# 6.2 Классификация.Фильмы

Для тех, кто хочет поставить точку в рекомендательных системах.

### Рекомендательная система и kNN

В прошлый раз мы использовали kMeans, тоже метрический алгоритм, но кластеризации, для построения "рекомендательной ситсемы". На самом деле тогда вы строили кластеры жанров/фильмов на основе предпочтений пользователей.  

Теперь попробуем kNN для рекомендаций. Мы построим модель, в которой не будет кластеров, но которая сможет выводить список наиболее похожих фильмов для заданного.

Подготовка датасета похожа на то, что мы уже делали. Отличаться будет только последний пункт, когда мы запустим kNN с косинусной метрикой.

Будем использовать известный нам [датасет с оценками фильмов](http://www.cs.umn.edu/GroupLens). Только в этот раз мы не будем кластеризовывать жанры, а будем рекомендовать похожие фильмы, основываясь на оценках других пользователей. То есть мы будем по факту предсказывать предпочтение нового пользователя на основе других.

Для рекомендательной системы в датасете в объектами будут пользователи, а признаками - оценки фильма данным пользователем.

![](https://i.imgur.com/hn3xjZl.png)

Мы будем использовать метрический алгоритм для поиска ближайших соседей, тем самым рекомендовать фильмы близкие к данному.

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
movies = pd.read_csv('../Data/imdb_movies.csv.gz', index_col='movieId')
ratings = pd.read_csv('../Data/imdb_ratings.csv.gz')

print(movies.shape)
print(ratings.shape)

(9125, 2)
(100004, 4)


In [3]:
movies.head()

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [4]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


### 1.1 Поиск популярных фильмов (3 балла)

Среди всех фильмов есть не популярные, для которых выставлена 1-2 оценки. Такого количества данных не достаточно, поэтому нужно срезать много фильмов с маленьким числом отзывов.

Посчтитайте количество оценок для каждого фильма.

In [5]:
rates_count = ratings.groupby('movieId').count().rename(columns={"userId": "rates_count"}).drop(['rating', 'timestamp'], axis=1)
rates_count

,rates_count
movieId,
1,247
2,107
3,59
4,13
5,56
...,...
161944,1
162376,1
162542,1


Выведите сводную статистику по количеству оценок фильмов. (общее число фильмов, среднее количество оценок, дисперсию, квантили)

In [6]:
rates_count.describe()

,rates_count
count,9066.000000
mean,11.030664
std,24.050800
min,1.000000
25%,1.000000
50%,3.000000
75%,9.000000
max,341.000000


Как мы видим `75%` квантиль показывет, что `75%` фильмов имело очень мало оценок. Нам столько будет недостаточно. Посчтитайте квантили в диапазоне от `75%` до `100%` с шагом в `1%`. Выберите тот квантиль, при котором количество оценок равно 20. Напишите какой это квантиль. Напишите сколько всего фильмов останется, если отбросить все фильмы у которых рейтинг меньше этого квантиля.

In [7]:
for i in range(25):
    print("Quantile {}% value {}".format(75+i,rates_count.quantile((75+i)/100)['rates_count']))

Quantile 75% value 9.0
Quantile 76% value 10.0
Quantile 77% value 10.0
Quantile 78% value 11.0
Quantile 79% value 12.0
Quantile 80% value 13.0
Quantile 81% value 14.0
Quantile 82% value 15.0
Quantile 83% value 16.0
Quantile 84% value 17.0
Quantile 85% value 19.0
Quantile 86% value 20.0
Quantile 87% value 22.0
Quantile 88% value 23.0
Quantile 89% value 25.0
Quantile 90% value 28.0
Quantile 91% value 31.0
Quantile 92% value 34.0
Quantile 93% value 38.0
Quantile 94% value 44.0
Quantile 95% value 49.0
Quantile 96% value 58.0
Quantile 97% value 69.04999999999927
Quantile 98% value 89.0
Quantile 99% value 123.0


In [8]:
good_quantile = 0.86
quantile_val = rates_count.quantile(good_quantile)['rates_count']
popular_movies = rates_count[rates_count['rates_count'] > quantile_val].join(movies, how='left')
print("Movies left {}".format(len(popular_movies)))
popular_movies

Movies left 1247


,rates_count,title,genres
movieId,,,
1,247,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,107,Jumanji (1995),Adventure|Children|Fantasy
3,59,Grumpier Old Men (1995),Comedy|Romance
5,56,Father of the Bride Part II (1995),Comedy
6,104,Heat (1995),Action|Crime|Thriller
...,...,...,...
116797,32,The Imitation Game (2014),Drama|Thriller|War
122882,31,Mad Max: Fury Road (2015),Action|Adventure|Sci-Fi|Thriller
122886,29,Star Wars: Episode VII - The Force Awakens (2015),Action|Adventure|Fantasy|Sci-Fi|IMAX


### 1.2 Соединяем таблицы (1 балл)

Будет немного не удобно искать фильмы по их идентификаторам, нам нужны названия. Поэтому соедините две таблицы `ratings` и `movies` по `movieId` и отфильтруйте отзывы на редкие фильмы, которые мы нашли в предыдущем шаге. Выведите первые 5 элементов полученного датасета.

In [9]:
mov_rat = ratings.join(popular_movies, on='movieId', how='inner')
mov_rat.head()

,userId,movieId,rating,timestamp,rates_count,title,genres
0,1,31,2.5,1260759144,42,Dangerous Minds (1995),Drama
498,7,31,3.0,851868750,42,Dangerous Minds (1995),Drama
6059,31,31,4.0,1273541953,42,Dangerous Minds (1995),Drama
6130,32,31,4.0,834828440,42,Dangerous Minds (1995),Drama
6526,36,31,3.0,847057202,42,Dangerous Minds (1995),Drama


### 1.3 Матрица Фильмы-Пользователи

В нашем случае мы будем искать похожие фильмы, поэтому сделайте из датасета оценок матрицу, где в строках фильмы, в колонках пользователи, а в ячейках оценки. Пропущенные значение, то есть отсутсвие оценки, заполните 0.

Для увеличения скорости выполнения алгоритма ближайших соседей примените функцию [`scipy.sparse.csc_matrix`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html),  которая создаст плотное представление нашей разреженой матрицы.

In [10]:
movie_uniq = list(mov_rat['title'].unique())
user_uniq = list(mov_rat['userId'].unique())
movie_indexes = { movie_uniq[i] : i for i in range(0, len(movie_uniq) ) }
user_indexes = { user_uniq[i] : i for i in range(0, len(user_uniq) ) }
mu_matrix = np.empty((len(movie_uniq), len(user_uniq)))
mu_df = pd.DataFrame(data=mu_matrix, index=movie_uniq, columns=user_uniq)
for index, row in mov_rat.iterrows():
    mu_df.at[row['title'], row['userId']] = row['rating']
mu_df

,1,7,31,32,36,39,73,88,96,110,...,542,6,526,204,221,45,289,444,227,301
Dangerous Minds (1995),2.5,3.0,4.0,4.0,3.0,3.0,3.5,3.0,2.5,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Dumbo (1941),3.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Sleepers (1996),3.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Escape from New York (1981),2.0,3.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Cinema Paradiso (Nuovo cinema Paradiso) (1989),4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Joy Luck Club, The (1993)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Opposite of Sex, The (1998)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Six Days Seven Nights (1998),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Money Pit, The (1986)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 1.4 Тренировка модели рекомендаций 

Так как по сути у нас нет каких-то меток классов, мы будем использовать алгоритма без учителя [`sklearn.neighbors.NearestNeighbors`](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html). Научите его по данным нашей матрицы пользователей-фильмов и напишите **функцию**, принимающую на вход название фильма и возвращающую список из 6 рекомендованых фильмов, начиная от более похожего.

Для определения "похожести" двух фильмов мы будем использовать косинусную меру, вместо евклидова или минковского:

$$\LARGE cos(\overrightarrow{x},\overrightarrow{y}) = \frac{\overrightarrow{x} \cdot \overrightarrow{y}}{||\overrightarrow{x}|| \times ||\overrightarrow{y}||}$$

In [11]:
from sklearn.neighbors import NearestNeighbors
nn = NearestNeighbors(n_neighbors=6, metric='cosine')
nn.fit(mu_df)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=6, p=2,
                 radius=1.0)

In [22]:
def similar(name='Pulp Fiction (1994)', n=6):
    # Тут какой-то код
    indices = nn.kneighbors(X = [mu_df.loc[name]], return_distance=False)
    print([mu_df.loc[name]])
    return [movie_uniq[i] for i in indices[0]]
    #return indices

Выведите рекомендации для фильмов:
- `Pulp Fiction (1994)`
- `Star Wars: Episode V - The Empire Strikes Back (1980)`
- `Lord of the Rings: The Two Towers, The (2002)`

In [23]:
similar('Pulp Fiction (1994)')

[1      0.0
7      0.0
31     4.5
32     2.0
36     4.0
      ... 
45     0.0
289    0.0
444    0.0
227    0.0
301    0.0
Name: Pulp Fiction (1994), Length: 671, dtype: float64]


['Pulp Fiction (1994)',
 'Silence of the Lambs, The (1991)',
 'Shawshank Redemption, The (1994)',
 'Seven (a.k.a. Se7en) (1995)',
 'Forrest Gump (1994)',
 'Usual Suspects, The (1995)']

In [14]:
similar('Star Wars: Episode V - The Empire Strikes Back (1980)')

['Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Star Wars: Episode VI - Return of the Jedi (1983)',
 'Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)',
 'Indiana Jones and the Last Crusade (1989)',
 'Back to the Future (1985)']

In [15]:
similar('Lord of the Rings: The Two Towers, The (2002)')

['Lord of the Rings: The Two Towers, The (2002)',
 'Lord of the Rings: The Return of the King, The (2003)',
 'Lord of the Rings: The Fellowship of the Ring, The (2001)',
 'Pirates of the Caribbean: The Curse of the Black Pearl (2003)',
 'Matrix, The (1999)',
 'Spider-Man (2002)']

### 1.5 Как посчитать качество рекомендации?

Как посчитать качество нашей классификации? У нас нет учителя, нет отложенной выборки. Мы просто посчитали все рейтинги и на основе них сделали предположения о похожести товаров. 

А как вы думаете можно проверить качество нашей рекомендательной системы? Напишите развернутый ответ своими словами.

Ну, вариантов много, зависит от желания и ресурсов:

1) Можно устроить массовое тестирование на людях и судить модель по их отзывам

2) Можно сравнить работу нашей модели с какой-то другой, которая зарекомендовала себя (зарекомендовала :D)

3) Можно прогнать через модель другую базу с фильмами и оценками, оценить работу (как часто людям нравятся фильмы, которые им рекомендуют)

4) Да даже на наших данных это можно в какой-то мере оценить.

![](https://69.media.tumblr.com/06a071083b3e62cd76f31af07ecb895f/tumblr_p49r4rWXG21uxovwqo1_540.gif)